In [17]:
import globus_sdk
import datetime
import time
import pytz
import pandas as pd
import calendar
import numpy as np

In [18]:
CLIENT_ID = 'ac34a1cf-7412-4f43-a693-935d7d09fd85'

client = globus_sdk.NativeAppAuthClient(CLIENT_ID)
client.oauth2_start_flow()

authorize_url = client.oauth2_get_authorize_url()
print('Please go to this URL and login: {0}'.format(authorize_url))

# this is to work on Python2 and Python3 -- you can just use raw_input() or
# input() for your specific version
get_input = getattr(__builtins__, 'raw_input', input)
auth_code = get_input(
    'Please enter the code you get after login here: ').strip()
token_response = client.oauth2_exchange_code_for_tokens(auth_code)

globus_auth_data = token_response.by_resource_server['auth.globus.org']
globus_transfer_data = token_response.by_resource_server['transfer.api.globus.org']

# most specifically, you want these tokens as strings
AUTH_TOKEN = globus_auth_data['access_token']
TRANSFER_TOKEN = globus_transfer_data['access_token']

Please go to this URL and login: https://auth.globus.org/v2/oauth2/authorize?client_id=ac34a1cf-7412-4f43-a693-935d7d09fd85&redirect_uri=https%3A%2F%2Fauth.globus.org%2Fv2%2Fweb%2Fauth-code&scope=openid+profile+email+urn%3Aglobus%3Aauth%3Ascope%3Atransfer.api.globus.org%3Aall&state=_default&response_type=code&code_challenge=JdFeV1glXKZKiZVC6A208locYa9qmwwazy0ERcWtyS0&code_challenge_method=S256&access_type=online
Please enter the code you get after login here: 4Biu0vAcW8Xg56tZHZLaseEAHlLR42


In [19]:
# a GlobusAuthorizer is an auxiliary object we use to wrap the token. In
# more advanced scenarios, other types of GlobusAuthorizers give us
# expressive power
authorizer = globus_sdk.AccessTokenAuthorizer(TRANSFER_TOKEN)
tc = globus_sdk.TransferClient(authorizer=authorizer)

# high level interface; provides iterators for list responses
print("My Endpoints:")
for ep in tc.endpoint_search(filter_scope="my-endpoints"):
    print("[{}] {}".format(ep["id"], ep["display_name"]))

My Endpoints:
[18c03e00-8157-11e8-94e4-0a6d4e044368] nersc_transfer


In [20]:
client = globus_sdk.NativeAppAuthClient(CLIENT_ID)
client.oauth2_start_flow()

print('Please go to this URL and login: {0}'
      .format(client.oauth2_get_authorize_url()))

get_input = getattr(__builtins__, 'raw_input', input)
auth_code = get_input('Please enter the code here: ').strip()
token_response = client.oauth2_exchange_code_for_tokens(auth_code)

Please go to this URL and login: https://auth.globus.org/v2/oauth2/authorize?client_id=ac34a1cf-7412-4f43-a693-935d7d09fd85&redirect_uri=https%3A%2F%2Fauth.globus.org%2Fv2%2Fweb%2Fauth-code&scope=openid+profile+email+urn%3Aglobus%3Aauth%3Ascope%3Atransfer.api.globus.org%3Aall&state=_default&response_type=code&code_challenge=BoX-cXkGxu7U_QvxSUHc6u8GfJdxluvIpX4r4kfi1so&code_challenge_method=S256&access_type=online
Please enter the code here: XsFiCotf62vyQRSOcVpKSe3JPFqspt


In [21]:
str(token_response.by_resource_server)

"{'auth.globus.org': {'scope': 'profile openid email', 'access_token': 'Ag80D1JG5jgvdO9kDqYmNXxybk0xQ4nGOQv8ybm4M633EDW3P9t8CJ9zedX4l5kVj4qVx9O4Dz1qBrh7Y47BgUB9d3U3pyQsKM43', 'refresh_token': None, 'token_type': 'Bearer', 'expires_at_seconds': 1537077647, 'resource_server': 'auth.globus.org'}, 'transfer.api.globus.org': {'scope': 'urn:globus:auth:scope:transfer.api.globus.org:all', 'access_token': 'AgkG4vQrmnJqbGwnVPoMn8ombpok0jKOzz2P6KYXqz3q2Wd2kMCOCG7OXnN7E4qdBMyPYmygvv67pFjx7jWbs6q0D', 'refresh_token': None, 'token_type': 'Bearer', 'expires_at_seconds': 1537077647, 'resource_server': 'transfer.api.globus.org'}}"

In [22]:
client = globus_sdk.NativeAppAuthClient(CLIENT_ID)
client.oauth2_start_flow(refresh_tokens=True)

print('Please go to this URL and login: {0}'
      .format(client.oauth2_get_authorize_url()))

get_input = getattr(__builtins__, 'raw_input', input)
auth_code = get_input('Please enter the code here: ').strip()
token_response = client.oauth2_exchange_code_for_tokens(auth_code)

Please go to this URL and login: https://auth.globus.org/v2/oauth2/authorize?client_id=ac34a1cf-7412-4f43-a693-935d7d09fd85&redirect_uri=https%3A%2F%2Fauth.globus.org%2Fv2%2Fweb%2Fauth-code&scope=openid+profile+email+urn%3Aglobus%3Aauth%3Ascope%3Atransfer.api.globus.org%3Aall&state=_default&response_type=code&code_challenge=ykvHf5bi7Ogtitxd1a_xarU6oalr40O0j-g7NeEo__A&code_challenge_method=S256&access_type=offline
Please enter the code here: pJ9o9OuLyqKMDrLxFua16wJWUawjvO


In [23]:
str(token_response.by_resource_server)

"{'auth.globus.org': {'scope': 'profile openid email', 'access_token': 'Ag6ylVq5YBQPNK5bWxme3GvaM97erEldPw42Xv9vv9j8Jmzm6ju2C9vaE6Gr9W090OyrJ8lpyGy6wGT14g1VKUW9eDSO58EuNQPN', 'refresh_token': 'Agd3No49lM3GgneVQJoMEKeQQYxOW44rWMbEGJNKo12ko1PVrPUOUgakwWpbMEvQd6pDV1awGQmvaJbrKEd8wBBDlDYB', 'token_type': 'Bearer', 'expires_at_seconds': 1537077663, 'resource_server': 'auth.globus.org'}, 'transfer.api.globus.org': {'scope': 'urn:globus:auth:scope:transfer.api.globus.org:all', 'access_token': 'AgP31olpxrr21GaKeJaJlXG2WOobMl3KKy6JMzD3lEk6p0aeOMFmCEq6YO57weOKDnln0k0J9yYMWDuqKWqvbI7xow', 'refresh_token': 'AgOGKzbDJDYqyMQnqQmJEQqv1YQ24XomdXoO2v7EnzraBlBNplIvU9wkO08vDw335NKgymYrpV4EXokQ2wdmXmexVPgOG', 'token_type': 'Bearer', 'expires_at_seconds': 1537077663, 'resource_server': 'transfer.api.globus.org'}}"

In [9]:
from globus_sdk import TransferClient

# you must have transfer_token in your config for this to work
#tc = TransferClient()

print("My Last 25 Tasks:")
# `filter` to get Delete Tasks (default is just Transfer Tasks)
for task in tc.task_list(num_results=1, filter="type:TRANSFER,DELETE"):
    print(task["task_id"], task["type"], task["status"], task['effective_bytes_per_second'])

My Last 25 Tasks:
c352fcce-b468-11e8-8241-0a3b7ca8ce66 TRANSFER FAILED 8932591813


In [15]:
tdata = globus_sdk.TransferData(tc, "9d6d994a-6d04-11e5-ba46-22000b92c6ec",
                                 "e09e65f5-6d04-11e5-ba46-22000b92c6ec",
                                 label="nogrowcheckplease")
                                 #sync_level="checksum")
#tdata.add_item("/source/path/dir/", "/dest/path/dir/", recursive=True)
tdata.add_item("/global/cscratch1/sd/yuanlai/gf_test/new_test_files_same_size/", "/projects/AMASE/lyl/new_test_files_all_ost/", recursive=True)
#tdata.add_item("/global/homes/y/yuanlai/workspace/DTNSpeedTest/mpi_fread_test/s_fname_list.txt", "/dev/null")
#tdata.add_item("/global/homes/y/yuanlai/workspace/DTNSpeedTest/mpi_fread_test/s_fname_list.txt", "/dev/null")
transfer_result = tc.submit_transfer(tdata)
print("task_id =", transfer_result["task_id"])

task_id = ae6f08a0-aedc-11e8-823c-0a3b7ca8ce66


In [25]:
nersc_id = "9d6d994a-6d04-11e5-ba46-22000b92c6ec"
alcf_id = "e09e65f5-6d04-11e5-ba46-22000b92c6ec"
from_big_dir = "/global/cscratch1/sd/yuanlai/gf_test/new_test_files_same_size_128/"
to_big_dir = "/projects/AMASE/lyl/new_test_files_same_size_128/"
from_small_dir = "/global/cscratch1/sd/yuanlai/gf_test/new_test_files_all_ost/"
to_small_dir = "/projects/AMASE/lyl/new_test_files_all_ost/"

In [26]:
tdata = globus_sdk.TransferData(tc, nersc_id, alcf_id)
tdata.add_item(from_big_dir, to_big_dir, recursive=True)
transfer_result = tc.submit_transfer(tdata)
print("task_id =", transfer_result["task_id"])

task_id = a11f2bd4-b7e3-11e8-8bfb-0a1d4c5c824a


In [15]:
now = datetime.datetime.utcnow()
ddl = now + datetime.timedelta(minutes=10)
tdata = globus_sdk.TransferData(tc, nersc_id, alcf_id, label="nogrowcheckplease", deadline=str(ddl))
for i in range(128):
    tdata.add_item("/dev/zero", "/dev/null")
transfer_result = tc.submit_transfer(tdata)
print("task_id =", transfer_result["task_id"])

task_id = c352fcce-b468-11e8-8241-0a3b7ca8ce66


In [16]:
tc.get_task("c352fcce-b468-11e8-8241-0a3b7ca8ce66")['effective_bytes_per_second']

8932591813

In [ ]:
(pd.to_datetime('2018-09-02T22:01:40.000Z') - pd.datetime.utcfromtimestamp(0)).total_seconds().__int__()

In [52]:
calendar.timegm(now.utctimetuple())

1535921413

In [29]:
tdata = globus_sdk.TransferData(tc, nersc_id, alcf_id)
tdata.add_item(from_small_dir, to_small_dir, recursive=True)
transfer_result = tc.submit_transfer(tdata)

In [27]:
big_transfer_speed = []
small_transfer_speed = []
for i in range(10):
    tdata = globus_sdk.TransferData(tc, nersc_id, alcf_id)
    tdata.add_item(from_big_dir, to_big_dir, recursive=True)
    transfer_result = tc.submit_transfer(tdata)
    while not tc.task_wait(transfer_result["task_id"], timeout=10):
        pass
    big_transfer_speed.append(tc.get_task(transfer_result["task_id"])['effective_bytes_per_second'])
    
    tdata = globus_sdk.TransferData(tc, nersc_id, alcf_id)
    tdata.add_item(from_small_dir, to_small_dir, recursive=True)
    transfer_result = tc.submit_transfer(tdata)
    while not tc.task_wait(transfer_result["task_id"], timeout=10):
        pass
    small_transfer_speed.append(tc.get_task(transfer_result["task_id"])['effective_bytes_per_second'])
    print(i)
print(big_transfer_speed)
print(small_transfer_speed)
np.savez("data/bigVSsmall_10_big-same-size_speeds.npz", name1 = big_transfer_speed, name2 = small_transfer_speed)


0
1
2
3
4
5
6
7
8
9
[7164910614, 7889985104, 8116119049, 8173385169, 7962041528, 7454176841, 7745259194, 8170994430, 8116277753, 8108372580]
[3325414515, 3086004630, 3498641434, 3431044729, 3561453825, 3561073032, 3597265600, 3779552583, 3273483071, 3631841668]


In [71]:
np.load("data/bigVSsmall_10_speeds.npz")["name1"]/np.load("data/bigVSsmall_10_speeds.npz")["name2"]

array([2.00492703, 1.73845353, 2.02000409, 2.09034961, 2.35632986,
       1.99736833, 1.94575394, 2.09725914, 2.18795697, 2.11784687])